In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import os
from PIL import Image

In [2]:
TEST_DATA_PATH =  "./test"
TRAIN_DATA_PATH = "./train"

In [3]:
# data transform, you can add different transform methods
img_size = 224
train_transform = transforms.Compose([
                                    transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

test_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor(),
                                    transforms.CenterCrop(10),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

dataset = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_transform)
test_data = datasets.ImageFolder(root=TEST_DATA_PATH,transform=test_transform)

# spilt data into training and validation
TOTAL_SIZE = len(dataset)
ratio = 0.8
train_len = round(TOTAL_SIZE * ratio)
val_len = round(TOTAL_SIZE * (1-ratio))

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])

# data loader, you can choose the input arguments by yourself

train_data_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=32, shuffle=True,  num_workers=4)
test_data_loader  = data.DataLoader(test_data, batch_size=8, shuffle=False, num_workers=4) 

print(dataset)
print(dataset.class_to_idx)

Dataset ImageFolder
    Number of datapoints: 1646
    Root Location: ./train
    Transforms (if any): Compose(
                             Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                         )
    Target Transforms (if any): None
{'actinic keratosis': 0, 'basal cell carcinoma': 1, 'melanoma': 2, 'nevus': 3, 'pigmented benign keratosis': 4}


In [4]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
model = torchvision.models.resnet18(pretrained = True)

In [6]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)

In [7]:
learning_rate=1e-2
epochs = 30

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
lr_sch = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
criterion = nn.CrossEntropyLoss()

In [8]:

min_val_loss = float("inf")

for epoch in range(1, epochs+1):
    model.train()
    total_train_loss = 0
    total_val_loss = 0
    train_hit = 0
    val_hit = 0
    
    for data, target in train_data_loader:
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()

        output=model(data)
        
        # loss function
        loss = criterion(output, target)

        total_train_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_hit += pred.eq(target.data.view_as(pred)).cpu().sum().item() 


        # do back propagation
        loss.backward()
        optimizer.step()
    lr_sch.step()    
    
    
    with torch.no_grad():
        model.eval()
        for data, target in val_data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            total_val_loss += F.cross_entropy(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            val_hit += pred.eq(target.data.view_as(pred)).cpu().sum().item() 
    
    avg_train_loss = total_train_loss/len(train_data_loader)
    avg_val_loss   = total_val_loss/len(val_data_loader)
    
    print('Epoch:%3d'%epoch
        , '|Train Loss:%8.4f'%(avg_train_loss)
        , '|Train Acc:%3.4f'%(train_hit/len(train_data_loader.dataset)*100.0)
        , '|Val Loss:%8.4f'%(avg_val_loss)
        , '|Val Acc:%3.4f'%(val_hit/len(val_data_loader.dataset)*100.0))
    
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        print("-------------saving model--------------")
        # save the model
        torch.save(model, "model.pth")
    

Epoch:  1 |Train Loss:  1.3516 |Train Acc:42.4450 |Val Loss:  0.8720 |Val Acc:69.9088
-------------saving model--------------
Epoch:  2 |Train Loss:  0.7753 |Train Acc:71.8299 |Val Loss:  0.6614 |Val Acc:75.6839
-------------saving model--------------
Epoch:  3 |Train Loss:  0.5861 |Train Acc:80.1822 |Val Loss:  0.5991 |Val Acc:73.5562
-------------saving model--------------
Epoch:  4 |Train Loss:  0.4174 |Train Acc:85.6492 |Val Loss:  0.5273 |Val Acc:77.8116
-------------saving model--------------
Epoch:  5 |Train Loss:  0.3039 |Train Acc:89.9772 |Val Loss:  0.5595 |Val Acc:75.6839
Epoch:  6 |Train Loss:  0.2397 |Train Acc:93.3181 |Val Loss:  0.4961 |Val Acc:79.0274
-------------saving model--------------
Epoch:  7 |Train Loss:  0.2009 |Train Acc:94.0774 |Val Loss:  0.5834 |Val Acc:75.3799
Epoch:  8 |Train Loss:  0.1765 |Train Acc:94.6090 |Val Loss:  0.5242 |Val Acc:77.2036
Epoch:  9 |Train Loss:  0.1524 |Train Acc:94.9127 |Val Loss:  0.5061 |Val Acc:76.5957
Epoch: 10 |Train Loss:  0.